### Land vs. Ocean

Purpose: Read in precalculated feedbacks and energy fluxes and determine the differences in AA drivers depending on surface type.

By: Ty Janoski

Updated: 09.02.21

In [1]:
# import statements
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats

%matplotlib inline
%config InlineBackend.figure_format = "pdf"

#### Land

In [2]:
# all files have the same path
path = '/dx05/janoski/d10/Arctic_Research/cesm-LE/spat_avg_feedbacks/all/b40.1850.cam5-lens.01.'

# read in each dataarray, _g = global average, _a = arctic average (>70N)
AHT_g = xr.open_dataarray(path+'glb_AHT.nc')
alb_g = xr.open_dataarray(path+'glb_alb_toa.nc')
clw_g = xr.open_dataarray(path+'glb_clw_toa.nc')
co2_g = xr.open_dataarray(path+'glb_CO2.nc')
csw_g = xr.open_dataarray(path+'glb_csw_toa.nc')
dEdt_g = xr.open_dataarray(path+'glb_dEdt.nc')
dTS_g = xr.open_dataarray(path+'glb_dTS.nc')
sfc_g = xr.open_dataarray(path+'glb_Fsfc.nc')
lap_g = xr.open_dataarray(path+'glb_lapse.nc')
plk_g = xr.open_dataarray(path+'glb_planck.nc')
qlw_g = xr.open_dataarray(path+'glb_qlw_toa.nc')
qsw_g = xr.open_dataarray(path+'glb_qsw_toa.nc')
sat_g_all = xr.open_dataarray(path+'glb_dSAT.nc')

# add SW and LW components of cloud and Q feedbacks
cld_g = clw_g + csw_g
q_g = qlw_g + qsw_g

# planck is local deviation from global avg:
plk_gprime = ((plk_g / dTS_g.mean(dim='ens')) -
         (plk_g / dTS_g.mean(dim='ens'))) * dTS_g.mean(dim='ens')

In [3]:
path = '/dx05/janoski/d10/Arctic_Research/cesm-LE/spat_avg_feedbacks/land_only/b40.1850.cam5-lens.01.'

# read in each dataarray, _g = global average, _a = arc average (>70N)
AHT_a = xr.open_dataarray(path+'arc_AHT.nc')
alb_a = xr.open_dataarray(path+'arc_alb_toa.nc')
clw_a = xr.open_dataarray(path+'arc_clw_toa.nc')
co2_a = xr.open_dataarray(path+'arc_CO2.nc')
csw_a = xr.open_dataarray(path+'arc_csw_toa.nc')
dEdt_a = xr.open_dataarray(path+'arc_dEdt.nc')
dTS_a = xr.open_dataarray(path+'arc_dTS.nc')
sfc_a = xr.open_dataarray(path+'arc_Fsfc.nc')
lap_a = xr.open_dataarray(path+'arc_lapse.nc')
plk_a = xr.open_dataarray(path+'arc_planck.nc')
qlw_a = xr.open_dataarray(path+'arc_qlw_toa.nc')
qsw_a = xr.open_dataarray(path+'arc_qsw_toa.nc')
sat_a_lnd = xr.open_dataarray(path+'arc_dSAT.nc')

# planck is local deviation from global avg:
plk_aprime = ((plk_a / dTS_a.mean(dim='ens')) -
         (plk_g / dTS_g.mean(dim='ens'))) * dTS_a.mean(dim='ens')

# add SW and LW components of cloud and Q feedbacks
cld_a = clw_a + csw_a
q_a = qlw_a + qsw_a

In [4]:
# To convert to temperature contributions, we need the planck feedback 
# for all land surfaces
path = ('/dx05/janoski/d10/Arctic_Research/cesm-LE/spat_avg_feedbacks/'
        'all/b40.1850.cam5-lens.01.')
norm = -1 * (xr.open_dataarray(path+'glb_planck.nc').mean(dim='ens') /
        xr.open_dataarray(path+'glb_dTS.nc').mean(dim='ens'))

In [5]:
AHT_diff = (AHT_a - AHT_g)/norm
AHT_mean = AHT_diff.mean(dim='ens').rolling(time=15,min_periods=1).mean()
AHT_p = stats.ttest_1samp(AHT_diff,0,axis=0)[1]

alb_diff = (alb_a - alb_g)/norm
alb_mean = alb_diff.mean(dim='ens')
alb_p = stats.ttest_1samp(alb_diff,0,axis=0)[1]

cld_diff = (cld_a - cld_g)/norm
cld_mean = cld_diff.mean(dim='ens')
cld_p = stats.ttest_1samp(cld_diff,0,axis=0)[1]

dEdt_diff = (dEdt_a - dEdt_g)/norm
dEdt_mean = dEdt_diff.mean(dim='ens').rolling(time=15,min_periods=1).mean()
dEdt_p = stats.ttest_1samp(dEdt_diff,0,axis=0)[1]

sfc_diff = (sfc_a - sfc_g)/norm
sfc_mean = sfc_diff.mean(dim='ens')
sfc_p = stats.ttest_1samp(sfc_diff,0,axis=0)[1]

lap_diff = (lap_a - lap_g)/norm
lap_mean = lap_diff.mean(dim='ens')
lap_p = stats.ttest_1samp(lap_diff,0,axis=0)[1]

plk_diff = (plk_aprime - plk_gprime)/norm
plk_mean = plk_diff.mean(dim='ens')
plk_p = stats.ttest_1samp(plk_diff,0,axis=0)[1]

q_diff = (q_a - q_g)/norm
q_mean = q_diff.mean(dim='ens')
q_p = stats.ttest_1samp(q_diff,0,axis=0)[1]

CO2_diff = np.tile((co2_a - co2_g),(2))/np.array(norm)

sat_diff_lnd = (sat_a_lnd - sat_g_all)
sat_mean_lnd = sat_diff_lnd.mean(dim='ens')
sat_p_lnd = stats.ttest_1samp(sat_diff_lnd,0,axis=0)[1]

In [6]:
# create figure object
fig, ax = plt.subplots(ncols=1,figsize=(5,4))

xs = np.arange(1,731,1)

first_days = np.array([1,91,182,274])

x_vals = np.concatenate((first_days,first_days+365))
x_labs = np.tile(['Jan 1','Apr 1','Jul 1','Oct 1'],
                 (2))

# plot 4xCO2 starting in Jan
ax.plot(xs, alb_mean, color='red',linestyle='dotted')
ax.plot(xs, alb_mean.where(alb_p<0.05), color='red',label='alb')
ax.plot(xs, AHT_mean, color='pink',linestyle='dotted')
ax.plot(xs, AHT_mean.where(AHT_p<0.05), color='pink',label='AHT')
ax.plot(xs, cld_mean, color='skyblue',linestyle='dotted')
ax.plot(xs, cld_mean.where(cld_p<0.05), color='skyblue',label='cloud')
# ax.plot(xs, dEdt_mean, color='greenyellow',linestyle='dotted')
# ax.plot(xs, dEdt_mean.where(dEdt_p<0.05), color='greenyellow',label='dE/dt')
ax.plot(xs, sfc_mean, color='navy',linestyle='dotted')
ax.plot(xs, sfc_mean.where(sfc_p<0.05), color='navy',label=r'F$_{sfc}$')
ax.plot(xs, lap_mean, color='green',linestyle='dotted')
ax.plot(xs, lap_mean.where(lap_p<0.05), color='green',label='LR')
ax.plot(xs, plk_mean, color='darkorange',linestyle='dotted')
ax.plot(xs, plk_mean.where(plk_p<0.05), color='darkorange',label='Planck')
ax.plot(xs, CO2_diff,color='gray',label='CO$_2$')
ax.set_xlim([1,731])

ax.tick_params(axis='x', rotation=45,labelsize=8)
ax.tick_params(axis='y',labelsize=8)
ax.set_ylabel(r'ΔT$_{s,Arc}$ - ΔT$_{s,global}$ (K)',fontsize=10)
ax.hlines(0,1,731,color='black',linestyle=':')
# ax[2,0].set_title(r'Change in SAT, 4xCO$_2$ in Jan, 30 ens. members',fontsize=10)
ax.legend(loc='best',fontsize=8,ncol=3)
ax.set_xticks(x_vals)
ax.set_xticklabels(x_labs)
ax.grid()

plt.title('Arctic land-only - global total')
plt.tight_layout()
plt.show()

<Figure size 360x288 with 1 Axes>

#### Ocean

In [7]:
path = '/dx05/janoski/d10/Arctic_Research/cesm-LE/spat_avg_feedbacks/ocean_only/b40.1850.cam5-lens.01.'

# read in each dataarray, _g = global average, _a = arctic average (>70N)
AHT_a = xr.open_dataarray(path+'arc_AHT.nc')
alb_a = xr.open_dataarray(path+'arc_alb_toa.nc')
clw_a = xr.open_dataarray(path+'arc_clw_toa.nc')
co2_a = xr.open_dataarray(path+'arc_CO2.nc')
csw_a = xr.open_dataarray(path+'arc_csw_toa.nc')
dEdt_a = xr.open_dataarray(path+'arc_dEdt.nc')
dTS_a = xr.open_dataarray(path+'arc_dTS.nc')
sfc_a = xr.open_dataarray(path+'arc_Fsfc.nc')
lap_a = xr.open_dataarray(path+'arc_lapse.nc')
plk_a = xr.open_dataarray(path+'arc_planck.nc')
qlw_a = xr.open_dataarray(path+'arc_qlw_toa.nc')
qsw_a = xr.open_dataarray(path+'arc_qsw_toa.nc')
sat_a_oce = xr.open_dataarray(path+'arc_dSAT.nc')

# planck is local deviation from global avg:
plk_aprime = ((plk_a / dTS_a.mean(dim='ens')) -
         (plk_g / dTS_g.mean(dim='ens'))) * dTS_a.mean(dim='ens')

# add SW and LW components of cloud and Q feedbacks
cld_a = clw_a + csw_a
q_a = qlw_a + qsw_a

In [8]:
# To convert to temperature contributions, we need the planck feedback for 
# all land surfaces
path = '/dx05/janoski/d10/Arctic_Research/cesm-LE/spat_avg_feedbacks/all/b40.1850.cam5-lens.01.'
norm = -1 * (xr.open_dataarray(path+'glb_planck.nc').mean(dim='ens') /
        xr.open_dataarray(path+'glb_dTS.nc').mean(dim='ens'))

In [9]:
AHT_diff = (AHT_a - AHT_g)/norm
AHT_mean = AHT_diff.mean(dim='ens').rolling(time=15,min_periods=1).mean()
AHT_p = stats.ttest_1samp(AHT_diff,0,axis=0)[1]

alb_diff = (alb_a - alb_g)/norm
alb_mean = alb_diff.mean(dim='ens')
alb_p = stats.ttest_1samp(alb_diff,0,axis=0)[1]

cld_diff = (cld_a - cld_g)/norm
cld_mean = cld_diff.mean(dim='ens')
cld_p = stats.ttest_1samp(cld_diff,0,axis=0)[1]

dEdt_diff = (dEdt_a - dEdt_g)/norm
dEdt_mean = dEdt_diff.mean(dim='ens').rolling(time=15,min_periods=1).mean()
dEdt_p = stats.ttest_1samp(dEdt_diff,0,axis=0)[1]

sfc_diff = (sfc_a - sfc_g)/norm
sfc_mean = sfc_diff.mean(dim='ens')
sfc_p = stats.ttest_1samp(sfc_diff,0,axis=0)[1]

lap_diff = (lap_a - lap_g)/norm
lap_mean = lap_diff.mean(dim='ens')
lap_p = stats.ttest_1samp(lap_diff,0,axis=0)[1]

plk_diff = (plk_aprime - plk_gprime)/norm
plk_mean = plk_diff.mean(dim='ens')
plk_p = stats.ttest_1samp(plk_diff,0,axis=0)[1]

q_diff = (q_a - q_g)/norm
q_mean = q_diff.mean(dim='ens')
q_p = stats.ttest_1samp(q_diff,0,axis=0)[1]

CO2_diff = np.tile((co2_a - co2_g),(2))/np.array(norm)

sat_diff_oce = (sat_a_oce - sat_g_all)
sat_mean_oce = sat_diff_oce.mean(dim='ens')
sat_p_oce = stats.ttest_1samp(sat_diff_oce,0,axis=0)[1]

In [10]:
# create figure object
fig, ax = plt.subplots(ncols=1,figsize=(5,4))

xs = np.arange(1,731,1)

first_days = np.array([1,91,182,274])

x_vals = np.concatenate((first_days,first_days+365))
x_labs = np.tile(['Jan 1','Apr 1','Jul 1','Oct 1'],
                 (2))

# plot 4xCO2 starting in Jan
ax.plot(xs, alb_mean, color='red',linestyle='dotted')
ax.plot(xs, alb_mean.where(alb_p<0.05), color='red',label='alb')
ax.plot(xs, AHT_mean, color='pink',linestyle='dotted')
ax.plot(xs, AHT_mean.where(AHT_p<0.05), color='pink',label='AHT')
ax.plot(xs, cld_mean, color='skyblue',linestyle='dotted')
ax.plot(xs, cld_mean.where(cld_p<0.05), color='skyblue',label='cloud')
# ax.plot(xs, dEdt_mean, color='coral',linestyle='dotted')
# ax.plot(xs, dEdt_mean.where(dEdt_p<0.05), color='coral',label='dE/dt')
ax.plot(xs, sfc_mean, color='navy',linestyle='dotted')
ax.plot(xs, sfc_mean.where(sfc_p<0.05), color='navy',label=r'F$_{sfc}$')
ax.plot(xs, lap_mean, color='green',linestyle='dotted')
ax.plot(xs, lap_mean.where(lap_p<0.05), color='green',label='LR')
ax.plot(xs, plk_mean, color='darkorange',linestyle='dotted')
ax.plot(xs, plk_mean.where(plk_p<0.05), color='darkorange',label='Planck')
ax.plot(xs, CO2_diff,color='gray',label='CO$_2$')
# ax.plot(xs, lr_diff, color='green',label='LR')
# ax.plot(xs, plk_a_norm, color='darkorange',label='Planck')
# # ax[2,0].set_ylim([-3,14])
ax.set_xlim([1,731])

ax.tick_params(axis='x', rotation=45,labelsize=8)
ax.tick_params(axis='y',labelsize=8)
ax.set_ylabel(r'ΔT$_{s,Arc}$ - ΔT$_{s,global}$ (K)',fontsize=10)
ax.hlines(0,1,731,color='black',linestyle=':')
# ax[2,0].set_title(r'Change in SAT, 4xCO$_2$ in Jan, 30 ens. members',fontsize=10)
ax.legend(loc='best',fontsize=8,ncol=3)
ax.set_xticks(x_vals)
ax.set_xticklabels(x_labs)
ax.grid()

plt.title('Arctic Ocean-only - Global Total')
plt.tight_layout()
plt.show()

<Figure size 360x288 with 1 Axes>

#### Land + Ocean

In [3]:
# all files have the same path
path = '/dx02/janoski/cesm/spat_avg_feedbacks/all/b40.1850.cam5-lens.01.'

# read in each dataarray, _g = global average, _a = arctic average (>70N)
AHT_g = xr.open_dataarray(path+'glb_AHT.nc')
alb_g = xr.open_dataarray(path+'glb_alb_toa.nc')
clw_g = xr.open_dataarray(path+'glb_clw_toa.nc')
co2_g = xr.open_dataarray(path+'glb_CO2.nc')
csw_g = xr.open_dataarray(path+'glb_csw_toa.nc')
dEdt_g = xr.open_dataarray(path+'glb_dEdt.nc')
dTS_g = xr.open_dataarray(path+'glb_dTS.nc')
sfc_g = xr.open_dataarray(path+'glb_Fsfc.nc')
lap_g = xr.open_dataarray(path+'glb_lapse.nc')
plk_g = xr.open_dataarray(path+'glb_planck.nc')
qlw_g = xr.open_dataarray(path+'glb_qlw_toa.nc')
qsw_g = xr.open_dataarray(path+'glb_qsw_toa.nc')
sat_g_all = xr.open_dataarray(path+'glb_dSAT.nc')

# add SW and LW components of cloud and Q feedbacks
cld_g = clw_g + csw_g
q_g = qlw_g + qsw_g

# planck is local deviation from global avg:
plk_gprime = ((plk_g / dTS_g.mean(dim='ens')) -
         (plk_g / dTS_g.mean(dim='ens'))) * dTS_g.mean(dim='ens')

In [4]:
# read in each dataarray, _g = global average, _a = arctic average (>70N)
AHT_a = xr.open_dataarray(path+'arc_AHT.nc')
alb_a = xr.open_dataarray(path+'arc_alb_toa.nc')
clw_a = xr.open_dataarray(path+'arc_clw_toa.nc')
co2_a = xr.open_dataarray(path+'arc_CO2.nc')
csw_a = xr.open_dataarray(path+'arc_csw_toa.nc')
dEdt_a = xr.open_dataarray(path+'arc_dEdt.nc')
dTS_a = xr.open_dataarray(path+'arc_dTS.nc')
sfc_a = xr.open_dataarray(path+'arc_Fsfc.nc')
lap_a = xr.open_dataarray(path+'arc_lapse.nc')
plk_a = xr.open_dataarray(path+'arc_planck.nc')
qlw_a = xr.open_dataarray(path+'arc_qlw_toa.nc')
qsw_a = xr.open_dataarray(path+'arc_qsw_toa.nc')
sat_a_all = xr.open_dataarray(path+'arc_dSAT.nc')


# planck is local deviation from global avg:
plk_aprime = ((plk_a / dTS_a.mean(dim='ens')) -
         (plk_g / dTS_g.mean(dim='ens'))) * dTS_a.mean(dim='ens')

# add SW and LW components of cloud and Q feedbacks
cld_a = clw_a + csw_a
q_a = qlw_a + qsw_a

In [6]:
# To convert to temperature contributions, we need the planck feedback for 
# all land surfaces
path = '/dx02/janoski/cesm/spat_avg_feedbacks/all/b40.1850.cam5-lens.01.'
norm = -1 * (xr.open_dataarray(path+'glb_planck.nc').mean(dim='ens') /
        xr.open_dataarray(path+'glb_dTS.nc').mean(dim='ens'))

In [7]:
AHT_diff = (AHT_a - AHT_g)/norm
AHT_mean = AHT_diff.mean(dim='ens').rolling(time=15,min_periods=1).mean()
AHT_p = stats.ttest_1samp(AHT_diff,0,axis=0)[1]

alb_diff = (alb_a - alb_g)/norm
alb_mean = alb_diff.mean(dim='ens')
alb_p = stats.ttest_1samp(alb_diff,0,axis=0)[1]

cld_diff = (cld_a - cld_g)/norm
cld_mean = cld_diff.mean(dim='ens')
cld_p = stats.ttest_1samp(cld_diff,0,axis=0)[1]

dEdt_diff = (dEdt_a - dEdt_g)/norm
dEdt_mean = dEdt_diff.mean(dim='ens').rolling(time=15,min_periods=1).mean()
dEdt_p = stats.ttest_1samp(dEdt_diff,0,axis=0)[1]

sfc_diff = (sfc_a - sfc_g)/norm
sfc_mean = sfc_diff.mean(dim='ens')
sfc_p = stats.ttest_1samp(sfc_diff,0,axis=0)[1]

lap_diff = (lap_a - lap_g)/norm
lap_mean = lap_diff.mean(dim='ens')
lap_p = stats.ttest_1samp(lap_diff,0,axis=0)[1]

plk_diff = (plk_aprime - plk_gprime)/norm
plk_mean = plk_diff.mean(dim='ens')
plk_p = stats.ttest_1samp(plk_diff,0,axis=0)[1]

q_diff = (q_a - q_g)/norm
q_mean = q_diff.mean(dim='ens')
q_p = stats.ttest_1samp(q_diff,0,axis=0)[1]

CO2_diff = np.tile((co2_a - co2_g),(2))/np.array(norm)

sat_diff_all = (sat_a_all - sat_g_all)
sat_mean_all = sat_diff_all.mean(dim='ens')
sat_p_all = stats.ttest_1samp(sat_diff_all,0,axis=0)[1]

In [8]:
# create figure object
fig, ax = plt.subplots(ncols=1,figsize=(7,3))

xs = np.arange(1,731,1)

first_days = np.array([1,91,182,274])

x_vals = np.concatenate((first_days,first_days+365))
x_labs = np.tile(['Jan 1','Apr 1','Jul 1','Oct 1'],
                 (2))

# plot 4xCO2 starting in Jan
ax.plot(xs, alb_mean, color='red',linestyle='dotted')
ax.plot(xs, alb_mean.where(alb_p<0.05), color='red',label='Alb')
ax.plot(xs, AHT_mean, color='pink',linestyle='dotted')
ax.plot(xs, AHT_mean.where(AHT_p<0.05), color='pink',label='AHT')
ax.plot(xs, cld_mean, color='skyblue',linestyle='dotted')
ax.plot(xs, cld_mean.where(cld_p<0.05), color='skyblue',label='cloud')
# ax.plot(xs, dEdt_mean, color='coral',linestyle='dotted')
# ax.plot(xs, dEdt_mean.where(dEdt_p<0.05), color='coral',label='dE/dt')
ax.plot(xs, sfc_mean, color='navy',linestyle='dotted')
ax.plot(xs, sfc_mean.where(sfc_p<0.05), color='navy',label=r'OHU')
ax.plot(xs, lap_mean, color='green',linestyle='dotted')
ax.plot(xs, lap_mean.where(lap_p<0.05), color='green',label='LR')
ax.plot(xs, plk_mean, color='darkorange',linestyle='dotted')
ax.plot(xs, plk_mean.where(plk_p<0.05), color='darkorange',label='P\'')
ax.plot(xs, q_mean, color='purple',linestyle='dotted')
ax.plot(xs, q_mean.where(q_p<0.05), color='purple',label=r'WV')
ax.plot(xs, CO2_diff,color='gray',label='CO$_2$')
# ax.plot(xs, lr_diff, color='green',label='LR')
# ax.plot(xs, plk_a_norm, color='darkorange',label='Planck')
# # ax[2,0].set_ylim([-3,14])
ax.set_xlim([1,731])

ax.tick_params(axis='x', rotation=45,labelsize=8)
ax.tick_params(axis='y',labelsize=8)
ax.set_ylabel(r'ΔT$_{s,Arc}$ - ΔT$_{s,global}$ (K)',fontsize=10)
ax.hlines(0,1,731,color='black',linestyle=':')
# ax[2,0].set_title(r'Change in SAT, 4xCO$_2$ in Jan, 30 ens. members',fontsize=10)
ax.legend(loc='best',fontsize=8,ncol=3)
ax.set_xticks(x_vals)
ax.set_xticklabels(x_labs)
ax.grid()

# plt.title('Land + Ocean')
plt.tight_layout()
plt.show()

<Figure size 504x216 with 1 Axes>

In [9]:
# create figure object
fig, ax = plt.subplots(ncols=1,figsize=(7,3))

xs = np.arange(1,731,1)

first_days = np.array([1,32,60,91])

x_vals = np.concatenate((first_days,first_days+365))
x_labs = np.tile(['Jan 1','Feb 1','Mar 1','Apr 1'],
                 (2))

# plot 4xCO2 starting in Jan
ax.plot(xs, alb_mean, color='red',linestyle='dotted')
ax.plot(xs, alb_mean.where(alb_p<0.05), color='red',label='alb')
ax.plot(xs, AHT_mean, color='pink',linestyle='dotted')
ax.plot(xs, AHT_mean.where(AHT_p<0.05), color='pink',label='AHT')
ax.plot(xs, cld_mean, color='skyblue',linestyle='dotted')
ax.plot(xs, cld_mean.where(cld_p<0.05), color='skyblue',label='cloud')
# ax.plot(xs, dEdt_mean, color='coral',linestyle='dotted')
# ax.plot(xs, dEdt_mean.where(dEdt_p<0.05), color='coral',label='dE/dt')
ax.plot(xs, sfc_mean, color='navy',linestyle='dotted')
ax.plot(xs, sfc_mean.where(sfc_p<0.05), color='navy',label=r'OHU')
ax.plot(xs, lap_mean, color='green',linestyle='dotted')
ax.plot(xs, lap_mean.where(lap_p<0.05), color='green',label='LR')
ax.plot(xs, plk_mean, color='darkorange',linestyle='dotted')
ax.plot(xs, plk_mean.where(plk_p<0.05), color='darkorange',label='Planck')
ax.plot(xs, q_mean, color='purple',linestyle='dotted')
ax.plot(xs, q_mean.where(q_p<0.05), color='purple',label=r'WV')
ax.plot(xs, CO2_diff,color='gray',label='CO$_2$')
# ax.plot(xs, lr_diff, color='green',label='LR')
# ax.plot(xs, plk_a_norm, color='darkorange',label='Planck')
# # ax[2,0].set_ylim([-3,14])

ax.tick_params(axis='x', rotation=45,labelsize=8)
ax.tick_params(axis='y',labelsize=8)
ax.set_ylabel(r'ΔT$_{s,Arc}$ - ΔT$_{s,global}$ (K)',fontsize=10)
ax.hlines(0,1,731,color='black',linestyle=':')
# ax[2,0].set_title(r'Change in SAT, 4xCO$_2$ in Jan, 30 ens. members',fontsize=10)
ax.legend(loc='best',fontsize=8,ncol=3)
ax.set_xticks(x_vals)
ax.set_xticklabels(x_labs)
ax.grid()

plt.xlim([1,91])
plt.ylim(-5,5)

# plt.title('Land + Ocean')
plt.tight_layout()
plt.show()

<Figure size 504x216 with 1 Axes>

In [10]:
AHT_diff = (AHT_a)/norm
AHT_mean = AHT_diff.mean(dim='ens').rolling(time=15,min_periods=1).mean()
AHT_p = stats.ttest_1samp(AHT_diff,0,axis=0)[1]

alb_diff = (alb_a)/norm
alb_mean = alb_diff.mean(dim='ens')
alb_p = stats.ttest_1samp(alb_diff,0,axis=0)[1]

cld_diff = (cld_a)/norm
cld_mean = cld_diff.mean(dim='ens')
cld_p = stats.ttest_1samp(cld_diff,0,axis=0)[1]

dEdt_diff = (dEdt_a)/norm
dEdt_mean = dEdt_diff.mean(dim='ens').rolling(time=15,min_periods=1).mean()
dEdt_p = stats.ttest_1samp(dEdt_diff,0,axis=0)[1]

sfc_diff = (sfc_a)/norm
sfc_mean = sfc_diff.mean(dim='ens')
sfc_p = stats.ttest_1samp(sfc_diff,0,axis=0)[1]

lap_diff = (lap_a)/norm
lap_mean = lap_diff.mean(dim='ens')
lap_p = stats.ttest_1samp(lap_diff,0,axis=0)[1]

plk_diff = (plk_aprime)/norm
plk_mean = plk_diff.mean(dim='ens')
plk_p = stats.ttest_1samp(plk_diff,0,axis=0)[1]

q_diff = (q_a)/norm
q_mean = q_diff.mean(dim='ens')
q_p = stats.ttest_1samp(q_diff,0,axis=0)[1]

CO2_diff = np.tile((co2_a),(2))/np.array(norm)

sat_diff_all = (sat_a_all)
sat_mean_all = sat_diff_all.mean(dim='ens')
sat_p_all = stats.ttest_1samp(sat_diff_all,0,axis=0)[1]

In [11]:
# create figure object
fig, ax = plt.subplots(ncols=1,figsize=(7,3.5))

xs = np.arange(1,731,1)

first_days = np.array([1,91,182,274])

x_vals = np.concatenate((first_days,first_days+365))
x_labs = np.tile(['Jan 1','Apr 1','Jul 1','Oct 1'],
                 (2))

# plot 4xCO2 starting in Jan
ax.plot(xs, alb_mean, color='red',linestyle='dotted')
ax.plot(xs, alb_mean.where(alb_p<0.05), color='red',label='alb')
ax.plot(xs, AHT_mean, color='pink',linestyle='dotted')
ax.plot(xs, AHT_mean.where(AHT_p<0.05), color='pink',label='AHT')
ax.plot(xs, cld_mean, color='skyblue',linestyle='dotted')
ax.plot(xs, cld_mean.where(cld_p<0.05), color='skyblue',label='cloud')
# ax.plot(xs, dEdt_mean, color='coral',linestyle='dotted')
# ax.plot(xs, dEdt_mean.where(dEdt_p<0.05), color='coral',label='dE/dt')
ax.plot(xs, sfc_mean, color='navy',linestyle='dotted')
ax.plot(xs, sfc_mean.where(sfc_p<0.05), color='navy',label=r'OHU')
ax.plot(xs, lap_mean, color='green',linestyle='dotted')
ax.plot(xs, lap_mean.where(lap_p<0.05), color='green',label='LR')
ax.plot(xs, plk_mean, color='darkorange',linestyle='dotted')
ax.plot(xs, plk_mean.where(plk_p<0.05), color='darkorange',label='Planck')
ax.plot(xs, q_mean, color='purple',linestyle='dotted')
ax.plot(xs, q_mean.where(q_p<0.05), color='purple',label=r'WV')
ax.plot(xs, CO2_diff,color='gray',label='CO$_2$')
# ax.plot(xs, lr_diff, color='green',label='LR')
# ax.plot(xs, plk_a_norm, color='darkorange',label='Planck')
# # ax[2,0].set_ylim([-3,14])
ax.set_xlim([1,731])

ax.tick_params(axis='x', rotation=45,labelsize=8)
ax.tick_params(axis='y',labelsize=8)
ax.set_ylabel(r'ΔT$_{s,Arc}$ (K)',fontsize=10)
ax.hlines(0,1,731,color='black',linestyle=':')
# ax[2,0].set_title(r'Change in SAT, 4xCO$_2$ in Jan, 30 ens. members',fontsize=10)
ax.legend(loc='best',fontsize=8,ncol=3)
ax.set_xticks(x_vals)
ax.set_xticklabels(x_labs)
ax.grid()

# plt.title('Land + Ocean')
plt.tight_layout()
plt.show()

<Figure size 504x252 with 1 Axes>

In [14]:
alb_diff = (alb_g)/norm
alb_mean = alb_diff.mean(dim='ens')
alb_p = stats.ttest_1samp(alb_diff,0,axis=0)[1]

cld_diff = (cld_g)/norm
cld_mean = cld_diff.mean(dim='ens')
cld_p = stats.ttest_1samp(cld_diff,0,axis=0)[1]

dEdt_diff = (dEdt_g)/norm
dEdt_mean = dEdt_diff.mean(dim='ens').rolling(time=15,min_periods=1).mean()
dEdt_p = stats.ttest_1samp(dEdt_diff,0,axis=0)[1]

sfc_diff = (sfc_g)/norm
sfc_mean = sfc_diff.mean(dim='ens')
sfc_p = stats.ttest_1samp(sfc_diff,0,axis=0)[1]

lap_diff = (lap_g)/norm
lap_mean = lap_diff.mean(dim='ens')
lap_p = stats.ttest_1samp(lap_diff,0,axis=0)[1]

plk_diff = (plk_gprime)/norm
plk_mean = plk_diff.mean(dim='ens')
plk_p = stats.ttest_1samp(plk_diff,0,axis=0)[1]

q_diff = (q_g)/norm
q_mean = q_diff.mean(dim='ens')
q_p = stats.ttest_1samp(q_diff,0,axis=0)[1]

CO2_diff = np.tile((co2_g),(2))/np.array(norm)

sat_diff_all = (sat_g_all)
sat_mean_all = sat_diff_all.mean(dim='ens')
sat_p_all = stats.ttest_1samp(sat_diff_all,0,axis=0)[1]

In [15]:
# create figure object
fig, ax = plt.subplots(ncols=1,figsize=(7,3.5))

xs = np.arange(1,731,1)

first_days = np.array([1,91,182,274])

x_vals = np.concatenate((first_days,first_days+365))
x_labs = np.tile(['Jan 1','Apr 1','Jul 1','Oct 1'],
                 (2))

# plot 4xCO2 starting in Jan
ax.plot(xs, alb_mean, color='red',linestyle='dotted')
ax.plot(xs, alb_mean.where(alb_p<0.05), color='red',label='alb')
ax.plot(xs, cld_mean, color='skyblue',linestyle='dotted')
ax.plot(xs, cld_mean.where(cld_p<0.05), color='skyblue',label='cloud')
# ax.plot(xs, dEdt_mean, color='coral',linestyle='dotted')
# ax.plot(xs, dEdt_mean.where(dEdt_p<0.05), color='coral',label='dE/dt')
ax.plot(xs, sfc_mean, color='navy',linestyle='dotted')
ax.plot(xs, sfc_mean.where(sfc_p<0.05), color='navy',label=r'OHU')
ax.plot(xs, lap_mean, color='green',linestyle='dotted')
ax.plot(xs, lap_mean.where(lap_p<0.05), color='green',label='LR')
ax.plot(xs, q_mean, color='purple',linestyle='dotted')
ax.plot(xs, q_mean.where(q_p<0.05), color='purple',label=r'WV')
ax.plot(xs, CO2_diff,color='gray',label='CO$_2$')
# ax.plot(xs, lr_diff, color='green',label='LR')
# ax.plot(xs, plk_a_norm, color='darkorange',label='Planck')
# # ax[2,0].set_ylim([-3,14])
ax.set_xlim([1,731])

ax.tick_params(axis='x', rotation=45,labelsize=8)
ax.tick_params(axis='y',labelsize=8)
ax.set_ylabel(r'ΔT$_{s,global}$ (K)',fontsize=10)
ax.hlines(0,1,731,color='black',linestyle=':')
# ax[2,0].set_title(r'Change in SAT, 4xCO$_2$ in Jan, 30 ens. members',fontsize=10)
ax.legend(loc='best',fontsize=8,ncol=3)
ax.set_xticks(x_vals)
ax.set_xticklabels(x_labs)
ax.set_ylim([-3,3])
ax.grid()

# plt.title('Land + Ocean')
plt.tight_layout()
plt.show()

<Figure size 504x252 with 1 Axes>

In [16]:
# create figure object
fig, ax = plt.subplots(ncols=1,figsize=(7,3.5))

xs = np.arange(1,731,1)

first_days = np.array([1,91,182,274])

x_vals = np.concatenate((first_days,first_days+365))
x_labs = np.tile(['Jan 1','Apr 1','Jul 1','Oct 1'],
                 (2))

# plot 4xCO2 starting in Jan
ax.plot(xs, alb_mean, color='red',linestyle='dotted')
ax.plot(xs, alb_mean.where(alb_p<0.05), color='red',label='alb')
ax.plot(xs, AHT_mean, color='pink',linestyle='dotted')
ax.plot(xs, AHT_mean.where(AHT_p<0.05), color='pink',label='AHT')
ax.plot(xs, cld_mean, color='skyblue',linestyle='dotted')
ax.plot(xs, cld_mean.where(cld_p<0.05), color='skyblue',label='cloud')
# ax.plot(xs, dEdt_mean, color='coral',linestyle='dotted')
# ax.plot(xs, dEdt_mean.where(dEdt_p<0.05), color='coral',label='dE/dt')
ax.plot(xs, sfc_mean, color='navy',linestyle='dotted')
ax.plot(xs, sfc_mean.where(sfc_p<0.05), color='navy',label=r'OHU')
ax.plot(xs, lap_mean, color='green',linestyle='dotted')
ax.plot(xs, lap_mean.where(lap_p<0.05), color='green',label='LR')
ax.plot(xs, plk_mean, color='darkorange',linestyle='dotted')
ax.plot(xs, plk_mean.where(plk_p<0.05), color='darkorange',label='Planck')
ax.plot(xs, q_mean, color='purple',linestyle='dotted')
ax.plot(xs, q_mean.where(q_p<0.05), color='purple',label=r'WV')
ax.plot(xs, CO2_diff,color='gray',label='CO$_2$')
# ax.plot(xs, lr_diff, color='green',label='LR')
# ax.plot(xs, plk_a_norm, color='darkorange',label='Planck')
# # ax[2,0].set_ylim([-3,14])
ax.set_xlim([1,731])

ax.tick_params(axis='x', rotation=45,labelsize=8)
ax.tick_params(axis='y',labelsize=8)
ax.set_ylabel(r'ΔT$_{s,Arc}$ - ΔT$_{s,global}$ (K)',fontsize=10)
ax.hlines(0,1,731,color='black',linestyle=':')
# ax[2,0].set_title(r'Change in SAT, 4xCO$_2$ in Jan, 30 ens. members',fontsize=10)
ax.legend(loc='best',fontsize=8,ncol=3)
ax.set_xticks(x_vals)
ax.set_xticklabels(x_labs)
ax.grid()

# plt.title('Land + Ocean')
plt.tight_layout()
plt.show()

<Figure size 504x252 with 1 Axes>

In [17]:
# create figure object
fig, ax = plt.subplots(ncols=2,nrows=3,figsize=(8,6),sharex=True,sharey=True)

xs = np.arange(1,731,1)

first_days = np.array([1,91,182,274])

x_vals = np.concatenate((first_days,first_days+365))
x_labs = np.tile(['Jan 1','Apr 1','Jul 1','Oct 1'],
                 (2))
ax[0,0].plot(xs, sat_g_lnd.mean(dim='ens'), color='blue',label='global')
ax[0,0].fill_between(xs, sat_g_lnd.mean(dim='ens')-sat_g_lnd.std(dim='ens'),
                     sat_g_lnd.mean(dim='ens')+sat_g_lnd.std(dim='ens'),
                     color='blue',alpha=0.4)
ax[0,0].plot(xs, sat_a_lnd.mean(dim='ens'), color='red',label='Arctic')
ax[0,0].fill_between(xs, sat_a_lnd.mean(dim='ens')-sat_a_lnd.std(dim='ens'),
                     sat_a_lnd.mean(dim='ens')+sat_a_lnd.std(dim='ens'),color='red',alpha=0.4)

ax[1,0].plot(xs, sat_g_oce.mean(dim='ens'), color='blue',label='global')
ax[1,0].fill_between(xs, sat_g_oce.mean(dim='ens')-sat_g_oce.std(dim='ens'),
                     sat_g_oce.mean(dim='ens')+sat_g_oce.std(dim='ens'),
                     color='blue',alpha=0.4)
ax[1,0].plot(xs, sat_a_oce.mean(dim='ens'), color='red',label='Arctic')
ax[1,0].fill_between(xs, sat_a_oce.mean(dim='ens')-sat_a_oce.std(dim='ens'),
                     sat_a_oce.mean(dim='ens')+sat_a_oce.std(dim='ens'),color='red',alpha=0.4)

ax[2,0].plot(xs, sat_g_all.mean(dim='ens'), color='blue',label='global')
ax[2,0].fill_between(xs, sat_g_all.mean(dim='ens')-sat_g_all.std(dim='ens'),
                     sat_g_all.mean(dim='ens')+sat_g_all.std(dim='ens'),
                     color='blue',alpha=0.4)
ax[2,0].plot(xs, sat_a_all.mean(dim='ens'), color='red',label='Arctic')
ax[2,0].fill_between(xs, sat_a_all.mean(dim='ens')-sat_a_all.std(dim='ens'),
                     sat_a_all.mean(dim='ens')+sat_a_all.std(dim='ens'),color='red',alpha=0.4)

# ===============================================================

ax[0,1].plot(xs, sat_mean_lnd.where(abs(sat_p_lnd) <= 0.05), color='black')
ax[0,1].plot(xs, sat_mean_lnd, color='black',linestyle=':')

ax[1,1].plot(xs, sat_mean_oce.where(abs(sat_p_oce) <= 0.05), color='black')
ax[1,1].plot(xs, sat_mean_oce, color='black',linestyle=':')

ax[2,1].plot(xs, sat_mean_all.where(abs(sat_p_all) <= 0.05), color='black')
ax[2,1].plot(xs, sat_mean_all, color='black',linestyle=':')

# ===============================================================

ax[2,0].set_xlim([1,731])
ax[2,0].set_xticks(x_vals)
ax[2,0].set_xticklabels(x_labs)

for a in ax.flatten():
    a.grid()
    a.tick_params(axis='x', rotation=45,labelsize=8)
    a.tick_params(axis='y',labelsize=8,labelleft=True)
    
ax[0,0].legend(loc='upper left',fontsize=8)
ax[0,0].set_ylabel('ΔSAT, land',fontsize=8)
ax[0,1].set_ylabel('ΔSAT$_{arc-glb}$, ocean',fontsize=8)
ax[1,0].set_ylabel('ΔSAT, land',fontsize=8)
ax[1,1].set_ylabel('ΔSAT$_{arc-glb}$, ocean',fontsize=8)
ax[2,0].set_ylabel('ΔSAT, land+ocean',fontsize=8)
ax[2,1].set_ylabel('ΔSAT$_{arc-glb}$, land+ocean',fontsize=8)

ax[0,0].set_ylim([-3,15])
ax[0,0].set_yticks(np.arange(-3,16,3))

plt.tight_layout()
plt.show()

NameError: name 'sat_g_lnd' is not defined

<Figure size 576x432 with 6 Axes>

In [18]:
# create figure object
fig, ax = plt.subplots(ncols=2,nrows=3,figsize=(8,6),sharex=True,sharey=True)

xs = np.arange(1,731,1)

first_days = np.array([1,91,182,274])

x_vals = np.concatenate((first_days,first_days+365))
x_labs = np.tile(['Jan 1','Apr 1','Jul 1','Oct 1'],
                 (2))
ax[0,0].plot(xs, sat_g_lnd.mean(dim='ens'), color='blue',label='global')
ax[0,0].fill_between(xs, sat_g_lnd.mean(dim='ens')-sat_g_lnd.std(dim='ens'),
                     sat_g_lnd.mean(dim='ens')+sat_g_lnd.std(dim='ens'),
                     color='blue',alpha=0.4)
ax[0,0].plot(xs, sat_a_lnd.mean(dim='ens'), color='red',label='Arctic')
ax[0,0].fill_between(xs, sat_a_lnd.mean(dim='ens')-sat_a_lnd.std(dim='ens'),
                     sat_a_lnd.mean(dim='ens')+sat_a_lnd.std(dim='ens'),color='red',alpha=0.4)

ax[1,0].plot(xs, sat_g_oce.mean(dim='ens'), color='blue',label='global')
ax[1,0].fill_between(xs, sat_g_oce.mean(dim='ens')-sat_g_oce.std(dim='ens'),
                     sat_g_oce.mean(dim='ens')+sat_g_oce.std(dim='ens'),
                     color='blue',alpha=0.4)
ax[1,0].plot(xs, sat_a_oce.mean(dim='ens'), color='red',label='Arctic')
ax[1,0].fill_between(xs, sat_a_oce.mean(dim='ens')-sat_a_oce.std(dim='ens'),
                     sat_a_oce.mean(dim='ens')+sat_a_oce.std(dim='ens'),color='red',alpha=0.4)

ax[2,0].plot(xs, sat_g_all.mean(dim='ens'), color='blue',label='global')
ax[2,0].fill_between(xs, sat_g_all.mean(dim='ens')-sat_g_all.std(dim='ens'),
                     sat_g_all.mean(dim='ens')+sat_g_all.std(dim='ens'),
                     color='blue',alpha=0.4)
ax[2,0].plot(xs, sat_a_all.mean(dim='ens'), color='red',label='Arctic')
ax[2,0].fill_between(xs, sat_a_all.mean(dim='ens')-sat_a_all.std(dim='ens'),
                     sat_a_all.mean(dim='ens')+sat_a_all.std(dim='ens'),color='red',alpha=0.4)

# ===============================================================

ax[0,1].plot(xs, sat_mean_lnd.where(abs(sat_p_lnd) <= 0.05), color='black')
ax[0,1].plot(xs, sat_mean_lnd, color='black',linestyle=':')

ax[1,1].plot(xs, sat_mean_oce.where(abs(sat_p_oce) <= 0.05), color='black')
ax[1,1].plot(xs, sat_mean_oce, color='black',linestyle=':')

ax[2,1].plot(xs, sat_mean_all.where(abs(sat_p_all) <= 0.05), color='black')
ax[2,1].plot(xs, sat_mean_all, color='black',linestyle=':')

# ===============================================================

ax[2,0].set_xlim([1,731])
ax[2,0].set_xticks(x_vals)
ax[2,0].set_xticklabels(x_labs)

for a in ax.flatten():
    a.grid()
    a.tick_params(axis='x', rotation=45,labelsize=8)
    a.tick_params(axis='y',labelsize=8,labelleft=True)
    
ax[0,0].legend(loc='upper left',fontsize=8)
ax[0,0].set_ylabel('ΔSAT, land',fontsize=8)
ax[0,1].set_ylabel('ΔSAT$_{arc-glb}$, ocean',fontsize=8)
ax[1,0].set_ylabel('ΔSAT, land',fontsize=8)
ax[1,1].set_ylabel('ΔSAT$_{arc-glb}$, ocean',fontsize=8)
ax[2,0].set_ylabel('ΔSAT, land+ocean',fontsize=8)
ax[2,1].set_ylabel('ΔSAT$_{arc-glb}$, land+ocean',fontsize=8)

ax[0,0].set_ylim([-3,15])
ax[0,0].set_yticks(np.arange(-3,16,3))

plt.tight_layout()
plt.show()

NameError: name 'sat_g_lnd' is not defined

<Figure size 576x432 with 6 Axes>

In [19]:
# get Arctic feedbacks for M1 and 2Y

norm_co2 = np.array(norm.isel(time=slice(None,365)))

LR_AM1 = (lap_a/norm).isel(time=slice(None,31))
PL_AM1 = (plk_aprime/norm).isel(time=slice(None,31))
Q_AM1 = ((qlw_a+qsw_a)/norm).isel(time=slice(None,31))
Alb_AM1 = (alb_a/norm).isel(time=slice(None,31))
Cld_AM1 = ((clw_a + csw_a)/norm).isel(time=slice(None,31))
OHU_AM1 = ((sfc_a/norm)).isel(time=slice(None,31))
AHT_AM1 = ((AHT_a/norm)).isel(time=slice(None,31))
CO2_AM1 = ((co2_a/norm_co2)).isel(time=slice(None,31))

LR_AY2 = lap_a/norm
PL_AY2 = plk_aprime/norm
Q_AY2 = ((qlw_a+qsw_a)/norm)
Alb_AY2 = alb_a/norm
Cld_AY2 = (clw_a + csw_a)/norm
OHU_AY2 = sfc_a/norm
AHT_AY2 = AHT_a/norm
CO2_AY2 = ((co2_a/norm_co2))

LR_GM1 = (lap_g/norm).isel(time=slice(None,31))
PL_GM1 = (plk_gprime/norm).isel(time=slice(None,31))
Q_GM1 = ((qlw_g+qsw_g)/norm).isel(time=slice(None,31))
Alb_GM1 = (alb_g/norm).isel(time=slice(None,31))
Cld_GM1 = ((clw_g + csw_g)/norm).isel(time=slice(None,31))
OHU_GM1 = ((sfc_g/norm)).isel(time=slice(None,31))
AHT_GM1 = ((AHT_g/norm)).isel(time=slice(None,31)) * 0
CO2_GM1 = ((co2_g/norm_co2)).isel(time=slice(None,31))

LR_GY2 = lap_g/norm
PL_GY2 = plk_gprime/norm
Q_GY2 = ((qlw_g+qsw_g)/norm)
Alb_GY2 = alb_g/norm
Cld_GY2 = (clw_g + csw_g)/norm
OHU_GY2 = sfc_g/norm
AHT_GY2 = AHT_g/norm * 0
CO2_GY2 = ((co2_g/norm_co2))

In [20]:
arc_means = [LR_AM1.mean(dim=['time','ens']),PL_AM1.mean(dim=['time','ens']),Q_AM1.mean(dim=['time','ens']),Alb_AM1.mean(dim=['time','ens']),Cld_AM1.mean(dim=['time','ens']),
            OHU_AM1.mean(dim=['time','ens'])]
glb_means = [LR_GM1.mean(dim=['time','ens']),PL_GM1.mean(dim=['time','ens']),Q_GM1.mean(dim=['time','ens']),Alb_GM1.mean(dim=['time','ens']),Cld_GM1.mean(dim=['time','ens']),
            OHU_GM1.mean(dim=['time','ens'])]
        
x = np.arange(len(arc_means))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots(figsize=(7,3.5))
rects1 = ax.bar(x - width/2, arc_means, width, label='Arctic',color='red')
rects1a = ax.bar(x[-1]+1 - width/2,AHT_AM1.mean(dim=['time','ens']),width,color='red',hatch='///')
rects1b = ax.bar(x[-1]+2 - width/2,CO2_AM1.mean(dim='time'),width,color='red')
rects2 = ax.bar(x + width/2, glb_means, width, label='global',color='blue')
rects2a = ax.bar(x[-1]+1 + width/2,AHT_GM1.mean(dim='time'),width,color='blue',hatch='//')
rects2b = ax.bar(x[-1]+2 + width/2,CO2_GM1.mean(dim='time'),width,color='blue')

plt.text(x[0]-width/2,-0.2,'LR')
plt.text(x[1]-width/2,-0.2,'P\'')
plt.text(x[2]-width/2,-0.2,'WV')
plt.text(x[3]-width/2,-0.2,'Alb')
plt.text(x[4]-width/2,-0.2,'Cld')
plt.text(x[5]-width/2,0.1,'OHU')
plt.text(x[5]+1-width/2,-0.2,'AHT')
plt.text(x[5]+2-width/2,-0.2,'CO2')


# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Warming Contribution (K)')
ax.set_title('First month after 4xCO$_2$')
ax.set_xticks(x)
ax.spines["bottom"].set_position(("data", 0))
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.set_xticks([])
# ax.set_xticklabels(labels)
ax.legend()
ax.grid()
ax.set_ylim([-2.3,2.3])

fig.tight_layout()
plt.show()

# plt.savefig('feedbacks_1M_bar_chart.svg')

<Figure size 504x252 with 1 Axes>

In [21]:
arc_means = [LR_AY2.mean(dim=['time','ens']),PL_AY2.mean(dim=['time','ens']),Q_AY2.mean(dim=['time','ens']),Alb_AY2.mean(dim=['time','ens']),Cld_AY2.mean(dim=['time','ens']),
            OHU_AY2.mean(dim=['time','ens'])]
glb_means = [LR_GY2.mean(dim=['time','ens']),PL_GY2.mean(dim=['time','ens']),Q_GY2.mean(dim=['time','ens']),Alb_GY2.mean(dim=['time','ens']),Cld_GY2.mean(dim=['time','ens']),
            OHU_GY2.mean(dim=['time','ens'])]
        
x = np.arange(len(arc_means))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots(figsize=(7,3.5))
rects1 = ax.bar(x - width/2, arc_means, width, label='Arctic',color='red')
rects1a = ax.bar(x[-1]+1 - width/2,AHT_AY2.mean(dim=['time','ens']),width,color='red')
rects1b = ax.bar(x[-1]+2 - width/2,CO2_AY2.mean(dim='time'),width,color='red')
rects2 = ax.bar(x + width/2, glb_means, width, label='global',color='blue')
rects2a = ax.bar(x[-1]+1 + width/2,AHT_GY2.mean(dim='time'),width,color='blue',hatch='//')
rects2b = ax.bar(x[-1]+2 + width/2,CO2_GY2.mean(dim='time'),width,color='blue')

plt.text(x[0]-width/2,-0.5,'LR')
plt.text(x[1]-width/2,-0.2,'P\'')
plt.text(x[2]-width/2,-0.2,'WV')
plt.text(x[3]-width/2,-0.2,'Alb')
plt.text(x[4]-width/2,-0.2,'Cld')
plt.text(x[5]-width/2,0.1,'OHU')
plt.text(x[5]+1-width/2,0.1,'AHT')
plt.text(x[5]+2-width/2,-0.2,'CO2')


# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Warming Contribution (K)')
ax.set_title('First 2 years after 4xCO$_2$')
ax.set_xticks(x)
ax.spines["bottom"].set_position(("data", 0))
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.set_xticks([])
# ax.set_xticklabels(labels)
ax.legend()
ax.grid()
ax.set_ylim([-2.3,2.3])

fig.tight_layout()
plt.show()

# plt.savefig('feedbacks_1M_bar_chart.svg')

<Figure size 504x252 with 1 Axes>